In [30]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
import os
from ultralytics import YOLO
fo.config.requirement_error_level = 2

In [19]:
recyclables = ["Bottle", "Book"]
hazardous = ["Mobile phone"]
kitchen = ["Banana", "Apple", "Orange"]
residual = ["Plastic bag", "Toilet paper", "Coffee cup"]
wanted_classes = recyclables + hazardous + kitchen + residual
parent_categories = {
    'Bottle': 'recyclables',
    'Book': 'recyclables',
    'Mobile phone': 'hazardous',
    'Banana': 'kitchen',
    'Apple': 'kitchen',
    'Orange': 'kitchen',
    'Plastic bag': 'residual',
    'Toilet paper': 'residual',
    'Coffee cup': 'residual',
}

# 只使用Fifty-One
- 主要用于推理和可视化
- 封装了模型+预处理+后处理
- 设计目的是在FiftyOne生态中使用

## 获取数据集

In [20]:
# dataset_train = fo.load_dataset("oi-trashes-train-1000")

## 获取预训练模型

In [21]:
# foz.list_zoo_models()

In [22]:
# model = foz.load_zoo_model("yolov8s-oiv7-torch")

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\liangjiaqi\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [28]:
# model_path = foz.download_zoo_model("yolov8s-oiv7-torch")

Model 'yolov8s-oiv7-torch' is already downloaded


(<fiftyone.zoo.models.ZooModel at 0x1f650f01f70>,
 'C:\\Users\\liangjiaqi\\fiftyone\\__models__\\yolov8s-oiv7.pt')

## 执行预测

In [23]:
# dataset_train.apply_model(model, label_field="yolov8_pred")

 100% |███████████████| 1000/1000 [6.5m elapsed, 0s remaining, 2.5 samples/s]       


## 查看结果

In [26]:
# session = fo.launch_app(dataset_train)

# 使用微调参数

In [31]:
model = YOLO("yolov8s-oiv7.pt")

In [32]:
# 开始微调训练
results = model.train(
    data='./export_oi/dataset.yaml',           # 数据集配置，告诉模型数据的组织结构、类别数量、类别名称等
    epochs=50,                  # 训练轮数，控制训练迭代次数，影响模型收敛程度
    imgsz=640,                  # 图像尺寸，自动将图像调整为640x640，保持宽高比并填充
    batch=16,                   # 批次大小，每次梯度更新使用的样本数量，影响训练稳定性和内存使用
    device='cpu',               # 使用CPU（如果有GPU改为0或[0,1,2,3]）
    workers=4,                  # 数据加载线程数，并行加载和预处理数据，提高训练效率
    lr0=0.01,                   # 初始学习率，控制参数更新步长，影响收敛速度和稳定性
    patience=10,                # 早停耐心值，如果验证集性能连续10个epoch没有提升，提前停止训练
    save=True,                  # 保存检查点，保存最佳模型和最后模型，便于恢复训练和部署
    pretrained=True,            # 从预训练模型初始化权重，加速收敛和提高性能
    optimizer='auto',           # YOLOv8会自动选择适合的优化器（通常是SGD或Adam）
    verbose=True,               # 输出详细的训练进度和指标
    project='garbage_detection',# 项目名称，组织训练结果的目录结构
    name='yolov8s_finetuned'    # 实验名称，标识当前训练实验，便于区分不同配置
)

Ultralytics 8.3.231  Python-3.12.8 torch-2.5.1 CPU (13th Gen Intel Core i5-1340P)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./export_oi/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-oiv7.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_finetuned, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=Tr

## 细致微调

In [35]:
def precise_garbage_detection_finetune(model_path, data_path):
    # 阶段1：只训练检测头（快速适应新类别）
    print("=== 阶段1: 检测头微调 (层22) ===")
    # 策略逻辑：
    # 🎯 目标：快速适应新的垃圾类别
    # 🔒 方法：冻结骨干网络，只训练检测头
    # 🚀 效果：快速收敛，避免破坏预训练特征
    model = YOLO(model_path)
    model.train(
        data=data_path,# 告诉模型数据的组织结构、类别数量、类别名称等
        epochs=15,  # 控制训练迭代次数，影响模型收敛程度
        imgsz=640,  # 自动将图像调整为640x640，保持宽高比并填充
        batch=16,   # 每次梯度更新使用的样本数量，影响训练稳定性和内存使用
        lr0=0.01,   # 学习率
        freeze=22,  # 冻结前22层（骨干+颈部），只训练检测头
        patience=5,
        save=True,
        project='garbage_detection_precise',
        name='stage1_head_only',
        # HSV颜色空间加强
        hsv_h=0.01, # 色调变化范围 ±1%
        hsv_s=0.5,  # 饱和度变化范围 ±50%  
        hsv_v=0.3,  # 明度变化范围 ±30%
        # 🎨 模拟不同光照条件下的垃圾
        # 🌈 增强模型对颜色变化的鲁棒性
        # 📸 适应褪色、脏污等真实垃圾情况

        # 几何变换增强
        translate=0.05, # 平移范围 ±5%
        scale=0.2,      # 缩放范围 ±20%
        fliplr=0.3,     # 水平翻转概率 30%
        # 📍 模拟垃圾在不同位置出现
        # 🔍 增强多尺度检测能力  
        # 🪞 增加数据多样性，防止过拟合
    )
    
    # 阶段2：微调颈部网络（特征融合层）
    print("=== 阶段2: 颈部网络微调 (层10-21) ===")
    # 策略逻辑：
    # 🎯 目标：优化特征融合能力
    # 🔒 方法：冻结浅层骨干，训练颈部+头部
    # 🚀 效果：适应垃圾的多尺度特性
    model = YOLO('garbage_detection_precise/stage1_head_only/weights/best.pt')
    model.train(
        data=data_path,
        epochs=20,
        imgsz=640,
        batch=16,
        lr0=0.005,
        freeze=10,  # 冻结前10层（骨干），训练颈部+头部
        patience=7,
        project='garbage_detection_precise',
        name='stage2_neck_tuning',
        # 增强数据增强
        mosaic=0.8,     # 马赛克增强概率 80%
        mixup=0.05,     # MixUp增强概率 5%
        hsv_h=0.015,
        hsv_s=0.6, 
        hsv_v=0.4,
        translate=0.1,
        scale=0.3,
    )
    
    # 阶段3：微调深层骨干网络
    print("=== 阶段3: 深层骨干微调 (层5-9) ===")
    # 策略逻辑：
    # 🎯 目标：微调深层特征提取器
    # 🔒 方法：只冻结最浅层，训练大部分网络
    # 🚀 效果：让模型学习垃圾特有的高级特征
    model = YOLO('garbage_detection_precise/stage2_neck_tuning/weights/best.pt')
    model.train(
        data=data_path,
        epochs=15,
        imgsz=640,
        batch=16,
        lr0=0.001,
        freeze=5,   # 冻结前5层（浅层骨干），训练深层骨干+颈部+头部
        patience=10,
        project='garbage_detection_precise',
        name='stage3_deep_backbone',
        # 更强数据增强
        mosaic=1.0,
        mixup=0.1,
        hsv_h=0.02,
        hsv_s=0.7, 
        hsv_v=0.5,
        translate=0.15,
        scale=0.4,
        fliplr=0.5,
    )
    
    # 阶段4：全网络精调（可选）
    print("=== 阶段4: 全网络精调 ===")
    # 策略逻辑：
    # 🎯 目标：最终精度优化
    # 🔒 方法：全网络微调，精细调整
    # 🚀 效果：达到最佳性能
    model = YOLO('garbage_detection_precise/stage3_deep_backbone/weights/best.pt')
    model.train(
        data=data_path,
        epochs=10,
        imgsz=640,
        batch=16,
        lr0=0.0005,
        freeze=0,   # 解冻所有层
        patience=15,
        project='garbage_detection_precise',
        name='stage4_full_finetune',
        # 最强数据增强
        mosaic=1.0,
        mixup=0.15,
        copy_paste=0.1, # 复制粘贴增强概率 10%
        hsv_h=0.025,
        hsv_s=0.8, 
        hsv_v=0.6,
        translate=0.2,
        scale=0.5,
    )
    
    return 'garbage_detection_precise/stage4_full_finetune/weights/best.pt'

In [36]:
final_model_path = precise_garbage_detection_finetune(model_path="yolov8s-oiv7.pt", data_path='./export_oi/dataset.yaml')

=== 阶段1: 检测头微调 (层22) ===
New https://pypi.org/project/ultralytics/8.3.232 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.231  Python-3.12.8 torch-2.5.1 CPU (13th Gen Intel Core i5-1340P)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./export_oi/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.3, flipud=0.0, format=torchscript, fraction=1.0, freeze=22, half=False, hsv_h=0.01, hsv_s=0.5, hsv_v=0.3, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-oiv7.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=stage1_head_only

### 模型分阶段预处理增强策略

1. 阶段1：保守增强

    目标：稳定训练，快速适应
    
    - hsv_h=0.01, hsv_s=0.5, hsv_v=0.3    # 温和的颜色变化
    - translate=0.05, scale=0.2           # 小幅几何变换
    - fliplr=0.3                          # 适中的翻转概率

    没有mosaic/mixup等复杂增强
2. 阶段2：中等增强

    目标：提高泛化能力

    - hsv_h=0.015, hsv_s=0.6, hsv_v=0.4   # 增强颜色变化
    - translate=0.1, scale=0.3            # 增加几何变换范围
    - mosaic=0.8, mixup=0.05              # 引入复杂增强
3. 阶段3：强增强

    目标：充分挖掘数据潜力

    - hsv_h=0.02, hsv_s=0.7, hsv_v=0.5    # 强颜色变化
    - translate=0.15, scale=0.4           # 大范围几何变换
    - fliplr=0.5                          # 高翻转概率
    - mosaic=1.0, mixup=0.1               # 高频复杂增强
4. 阶段4：最强增强

    目标：极限泛化性能

    - hsv_h=0.025, hsv_s=0.8, hsv_v=0.6   # 极限颜色变化
    - translate=0.2, scale=0.5            # 最大几何变换
    - mosaic=1.0, mixup=0.15              # 高频复杂增强
    - copy_paste=0.1                      # 引入复制粘贴增强

### 为什么这些参数适合垃圾检测：
- hsv增强 → 适应不同光照下的垃圾外观
- 几何变换 → 模拟垃圾的任意位置和方向
- mosaic → 模拟垃圾堆叠的复杂场景
- mixup → 处理类别不平衡和罕见组合

#### 关于为何图像测试时不需要图像预处理
以房价预测为例：

训练阶段:
  - 数值特征: 使用训练集均值标准差进行标准化
  - 类别特征: 使用训练集类别建立编码映射
  - 缺失值: 使用训练集统计量填充

测试阶段必须:
  - 数值特征: 使用相同的均值标准差
  - 类别特征: 使用相同的编码映射  
  - 缺失值: 使用相同的填充值


为什么？因为模型学的是"转换后的特征空间"

以YOLOv8垃圾检测为例：

训练阶段:
  - 基础适配: 尺寸调整640x640 + 归一化0-1
  - 数据增强: HSV变化 + 翻转 + 缩放 + Mosaic等

测试阶段只做:
  - 基础适配: 尺寸调整640x640 + 归一化0-1
  - 不做数据增强！

为什么？因为模型学的是"视觉特征识别能力"

### 分阶段学习率设计：
1. 阶段1: lr0=0.01   # 高学习率快速适应新类别
2. 阶段2: lr0=0.005  # 中等学习率精细调整
3. 阶段3: lr0=0.001  # 低学习率微调特征
4. 阶段4: lr0=0.0005 # 极低学习率最终优化